In [1]:
pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [2]:
import base64
import requests
import numpy as np
import gdown
from zipfile import ZipFile
import os
from matplotlib import pyplot as plt

import torch.nn as nn
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor,Lambda
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchsummary import summary
import time
import os
import copy

import pandas as pd
from torchvision.io import read_image
import re
import cv2
from skimage.transform import resize
from google.colab.patches import cv2_imshow
from tqdm import tqdm

# Loading Data & Preproccessing

In [102]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def donwload_to_file(grdive_path,file_name,output_path):
  """
  function for downloading zip files from google drive and save it to desired path on colab

  inputs:
    grdive_path: path of file we want to download (url should be in export=download format)
    file_name: name of file we want to download
    output_path:path we extract data into
  outputs:
    we dont have any output!

  """
  gdown.download(grdive_path, file_name)
  with ZipFile(file_name, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(output_path)




In [ ]:
# for train
donwload_to_file('https://drive.google.com/u/0/uc?id=1GAZgPpTUBSfhne-Tp0GDkvSHuq6EMMbj&export=download','train_ende.zip','/content/gdrive/MyDrive/Phase_0_images')

Downloading...
From: https://drive.google.com/u/0/uc?id=1GAZgPpTUBSfhne-Tp0GDkvSHuq6EMMbj&export=download
To: /content/train_ende.zip
100%|██████████| 2.90G/2.90G [00:17<00:00, 165MB/s]


In [ ]:
# renaming train folder name (it's train_ende) to just train so it is more consistent with other folders names
os.rename('/content/gdrive/MyDrive/Phase_0_images/train_ende','/content/gdrive/MyDrive/Phase_0_images/train')

In [ ]:
# for validation
donwload_to_file('https://drive.google.com/u/0/uc?id=12HM8uVNjFg-HRZ15ADue4oLGFAYQwvTA&export=download','dev.zip','/content/gdrive/MyDrive/Phase_0_images')

Downloading...
From: https://drive.google.com/u/0/uc?id=12HM8uVNjFg-HRZ15ADue4oLGFAYQwvTA&export=download
To: /content/dev.zip
100%|██████████| 638M/638M [00:12<00:00, 51.1MB/s]


In [ ]:
# for test
donwload_to_file('https://drive.google.com/u/0/uc?id=1B9ZFmSTqfTMaqJ15nQDrRNLqBvo-B39W&export=download','test.zip','/content/gdrive/MyDrive/Phase_0_images')

Downloading...
From: https://drive.google.com/u/0/uc?id=1B9ZFmSTqfTMaqJ15nQDrRNLqBvo-B39W&export=download
To: /content/test.zip
100%|██████████| 641M/641M [00:08<00:00, 73.5MB/s]


In [5]:
!git clone https://github.com/XL2248/MSCTD.git

Cloning into 'MSCTD'...
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1217 (delta 13), reused 7 (delta 3), pack-reused 1190
Receiving objects: 100% (1217/1217), 102.24 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (616/616), done.
Checking out files: 100% (934/934), done.


In [6]:
class MSTCDDate(Dataset):
   

  def __init__(self,image_path,txt_path,language,mode,image_format,scale, transform=None, target_transform=None):
     
      text_loader = language + '_' + mode + '.txt'
      sentiment_loader = 'sentiment_' + mode+'.txt'
      index_file = 'image_index_' + mode + '.txt'
      self.image_path = image_path
      self.transform = transform
      self.target_transform = target_transform
      self.mode = mode
      self.image_format = image_format
      self.scale = scale
      for file_names in os.scandir(txt_path):
        name_file = os.path.basename(file_names)
        if name_file == text_loader:
          with open(file_names.path) as f:
            self.text_file = [line.rstrip('\n') for line in f]
        if name_file == sentiment_loader:
          with open(file_names.path) as f:
            self.sentiment_file = [line.rstrip('\n') for line in f]
        if name_file == index_file:
          with open(file_names.path) as f:
            self.index_file = [line.rstrip('\n') for line in f]

  def __len__(self):
      return len(self.text_file)

  def __getitem__(self, idx):
      image_path_complete = self.image_path + '/' + self.mode + '/'
      text_file_chosen = self.text_file[idx]
      sentiment_chosen = int(self.sentiment_file[idx])
      temp_image = cv2.imread(image_path_complete+str(idx)+ '.' + self.image_format)
      n1,n2,n3 = np.shape(temp_image)
      if self.scale != 1:
        image = resize(temp_image,[int(n1/self.scale),int(n2/self.scale)], anti_aliasing=True)
      else:
        image = temp_image
      if self.transform:
          image = self.transform(image)
      if self.target_transform:
          label = self.target_transform(sentiment_chosen)
      return  text_file_chosen,sentiment_chosen,image

In [7]:
import torchvision.transforms as transforms
transformation= transforms.Compose([transforms.ToTensor(),transforms.Resize((50,100)),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [8]:
train_dataset= MSTCDDate('/content/gdrive/MyDrive/Phase_0_images' ,'/content/MSCTD/MSCTD_data/ende','english','train','jpg',1,  transform = transformation)

In [9]:
test_dataset= MSTCDDate('/content/gdrive/MyDrive/Phase_0_images' ,'/content/MSCTD/MSCTD_data/ende','english','test','jpg',1,  transform = transformation)

In [10]:
val_dataset= MSTCDDate('/content/gdrive/MyDrive/Phase_0_images' ,'/content/MSCTD/MSCTD_data/ende','english','dev','jpg',1,  transform = transformation)

In [11]:
dataset_sizes = {'train':len(train_dataset),'test':len(test_dataset),'val':len(val_dataset)}

In [16]:
#creating a list of Train data for faster loading
train_data_set=[]
for a in train_dataset:
  train_data_set.append([a[2],a[1]])

In [ ]:
#creating a list of Test data for faster loading
test_data_set=[]
for a in test_dataset:
  test_data_set.append([a[2],a[1]])

In [19]:
#creating a list of Val data for faster loading
val_data_set=[]
for a in val_dataset:
  val_data_set.append([a[2],a[1]])

In [21]:
#Creating Batch
from torch.utils.data import DataLoader
train_data_loader= DataLoader(train_data_set,batch_size=64,shuffle = True)
test_data_loader= DataLoader(test_data_set,batch_size=64,shuffle = True)
val_data_loader= DataLoader(val_data_set,batch_size=64,shuffle = True)

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Loading Pretrained Models

In [23]:
#Loading pre Trained Resnet50
model_resnet = torchvision.models.resnet50(pretrained=True)
for param in model_resnet.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 3) # we have 3 output class

model_resnet = model_resnet.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [95]:
#Loading pre Trained mobilenet_v3
model_mobilenetv3 = torchvision.models.mobilenet_v3_large(pretrained=True)
for param in model_mobilenetv3.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model_mobilenetv3.classifier[3] = nn.Linear(1280, 3) # we have 3 output class

model_mobilenetv3 = model_mobilenetv3.to(device)

In [81]:
#Loading pre Trained VGG16
model_vgg = torchvision.models.vgg16(pretrained=True)
for param in model_vgg.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model_vgg.classifier[6] = nn.Linear(4096, 3) # we have 3 output class
model_vgg = model_vgg.to(device)

In [96]:
def train_model_manual(model, criterion, optimizer,scheduler,val_beark, num_epochs=25):
  """
  function for train our model! in this function we use dataloader directly. so we only can use it for our CIFAR10 dataset with our input name.
  inputs: 
      model: input model 
      criterion: desired loss function
      optimizer: our optimizer(!)
      scheduler: for changing learning rate after sum epochs
      num_epochs: number of epoches
      val_beark: threshold for early stopping, if after "val_beark" steps our model don't get better, we end procces
  output:
      model: our trained model!


  """
  train_acc = []
  train_loss = []
  test_acc = []
  test_loss = []
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  best_loss = 100000000000
  counter_val_beark = 0
  for epoch in range(num_epochs):
    ### Training
    model.train()
    loss_train = 0
    acc_train = 0
    counter = 1
    for TK in train_data_loader:
      # 1. Forward pass
      batch = TK[0]
      label = TK[1]
      batch = batch.to(device)
      label = label.to(device)
      outputs = model(batch) # model outputs raw logits 
      _, preds = torch.max(outputs, 1)

      # print(y_logits)
      # 2. Calculate loss and accuracy
      counter = counter + 1
      loss = criterion(outputs, label)
      acc_train += torch.sum(preds == label.data)
      loss_train = loss_train + loss
      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backwards
      loss.backward()

      # 5. Optimizer step
      optimizer.step()
    loss_train = loss_train/counter
    acc_train = acc_train/dataset_sizes['train']
    train_acc.append(acc_train)
    train_loss.append(loss_train)
    scheduler.step()
    ### Testing
    loss_test = 0
    acc_test = 0
    counter = 1
    model.eval()
    with torch.inference_mode():
      for TK in val_data_loader:
      # 1. Forward pass
        batch = TK[0]
        label = TK[1]
        batch = batch.to(device)
        label = label.to(device)
        output = model(batch) # model outputs raw logits 
        _, preds = torch.max(output, 1)
        loss_test= criterion(output, label.data)
        acc_test += torch.sum(preds == label.data)
      loss_test = loss_test/counter
      acc_test = acc_test/dataset_sizes['val']
      test_loss.append(loss_test)
      test_acc.append(acc_test)
    if acc_test > best_acc:
                best_acc = acc_test
                best_model_wts = copy.deepcopy(model.state_dict())
    if loss_test <= best_loss:
                best_loss = loss_test
                counter_val_beark = 0
    if loss_test > best_loss:
                counter_val_beark = counter_val_beark + 1
                if (counter_val_beark > val_beark):
                  print(f"early stopping happend!")
                  break;

    # Print out what's happening
    if epoch % 1 == 0:
      print(f"Epoch: {epoch} | Loss: {loss_train:.5f}, Acc: {acc_train:.2f}% | Test Loss: {loss_test:.5f}, Test Acc: {acc_test:.2f}%") 
  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best val Acc: {best_acc:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model,best_loss,best_acc

# Fine Tunning Pre Trained Models

In [32]:
#Fine Tunning Resnet50
criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_resnet.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=50, gamma=0.5)

model_resnet = model_resnet.to(device)
model_resnet_trained,best_loss,best_acc = train_model_manual(model_part_2, criterion, optimizer_conv,exp_lr_scheduler,20,
                         num_epochs=5)

Epoch: 0 | Loss: 1.09783, Acc: 0.39% | Test Loss: 1.21394, Test Acc: 0.32%
Epoch: 1 | Loss: 1.08603, Acc: 0.40% | Test Loss: 1.14618, Test Acc: 0.35%
Epoch: 2 | Loss: 1.07884, Acc: 0.41% | Test Loss: 1.18054, Test Acc: 0.36%
Epoch: 3 | Loss: 1.07899, Acc: 0.41% | Test Loss: 1.24866, Test Acc: 0.36%
Epoch: 4 | Loss: 1.07519, Acc: 0.42% | Test Loss: 1.00446, Test Acc: 0.36%
Training complete in 0m 53s
Best val Acc: 0.363223


In [98]:
#Fine Tunning mobilenetv3
criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_mobilenetv3.classifier[3].parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=50, gamma=0.5)

model_mobilenetv3 = model_mobilenetv3.to(device)
model_mobilenetv3_trained,best_loss,best_acc = train_model_manual(model_mobilenetv3, criterion, optimizer_conv,exp_lr_scheduler,20,
                         num_epochs=5)

Epoch: 0 | Loss: 1.14643, Acc: 0.36% | Test Loss: 1.10467, Test Acc: 0.36%
Epoch: 1 | Loss: 1.10701, Acc: 0.39% | Test Loss: 1.07378, Test Acc: 0.35%
Epoch: 2 | Loss: 1.10185, Acc: 0.40% | Test Loss: 1.07485, Test Acc: 0.36%
Epoch: 3 | Loss: 1.10101, Acc: 0.40% | Test Loss: 1.10958, Test Acc: 0.36%
Epoch: 4 | Loss: 1.09823, Acc: 0.40% | Test Loss: 1.03149, Test Acc: 0.36%
Training complete in 0m 24s
Best val Acc: 0.364408


In [82]:
#Fine Tunning vgg16
criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_vgg.classifier[6].parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=50, gamma=0.5)

model_vgg = model_vgg.to(device)
model_vgg_trained,best_loss,best_acc = train_model_manual(model_vgg, criterion, optimizer_conv,exp_lr_scheduler,20,
                         num_epochs=5)

Epoch: 0 | Loss: 1.31329, Acc: 0.36% | Test Loss: 1.17500, Test Acc: 0.33%
Epoch: 1 | Loss: 1.31745, Acc: 0.37% | Test Loss: 1.00009, Test Acc: 0.35%
Epoch: 2 | Loss: 1.32392, Acc: 0.37% | Test Loss: 0.79377, Test Acc: 0.37%
Epoch: 3 | Loss: 1.31586, Acc: 0.38% | Test Loss: 1.17152, Test Acc: 0.34%
Epoch: 4 | Loss: 1.31931, Acc: 0.37% | Test Loss: 1.20625, Test Acc: 0.37%
Training complete in 1m 31s
Best val Acc: 0.367569


Based on robustness of results, we choose VGG16 for training

In [99]:
#Loading pre Trained VGG16 Again!

model_vgg = torchvision.models.vgg16(pretrained=True)
for param in model_vgg.parameters():
    param.requires_grad = False


model_vgg.classifier[6] = nn.Linear(4096, 3) # we have 3 output class
model_vgg = model_vgg.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [100]:
#Training VGG16 for 100 Epochs
criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_vgg.classifier[6].parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=50, gamma=0.5)

model_vgg = model_vgg.to(device)
model_vgg_trained,best_loss,best_acc = train_model_manual(model_vgg, criterion, optimizer_conv,exp_lr_scheduler,20,
                         num_epochs=100)

Epoch: 0 | Loss: 1.32638, Acc: 0.35% | Test Loss: 0.90314, Test Acc: 0.37%
Epoch: 1 | Loss: 1.31605, Acc: 0.37% | Test Loss: 1.15904, Test Acc: 0.37%
Epoch: 2 | Loss: 1.30484, Acc: 0.37% | Test Loss: 0.86959, Test Acc: 0.37%
Epoch: 3 | Loss: 1.32128, Acc: 0.37% | Test Loss: 1.14432, Test Acc: 0.35%
Epoch: 4 | Loss: 1.31786, Acc: 0.37% | Test Loss: 1.15239, Test Acc: 0.33%
Epoch: 5 | Loss: 1.32229, Acc: 0.38% | Test Loss: 0.92662, Test Acc: 0.36%
Epoch: 6 | Loss: 1.31652, Acc: 0.38% | Test Loss: 1.29277, Test Acc: 0.33%
Epoch: 7 | Loss: 1.31066, Acc: 0.38% | Test Loss: 1.25141, Test Acc: 0.37%
Epoch: 8 | Loss: 1.31859, Acc: 0.37% | Test Loss: 1.29200, Test Acc: 0.37%
Epoch: 9 | Loss: 1.32126, Acc: 0.37% | Test Loss: 1.07426, Test Acc: 0.36%
Epoch: 10 | Loss: 1.32192, Acc: 0.38% | Test Loss: 1.25647, Test Acc: 0.34%
Epoch: 11 | Loss: 1.31920, Acc: 0.37% | Test Loss: 0.87483, Test Acc: 0.36%
Epoch: 12 | Loss: 1.30142, Acc: 0.38% | Test Loss: 1.08224, Test Acc: 0.36%
Epoch: 13 | Loss: 1.30

In [105]:
torch.save(model_vgg_trained.state_dict(), '/content/gdrive/MyDrive/gdrivemodel_vgg_trained_weights.pth')